In [ ]:
import pandas as pd
import os

In [ ]:
os.chdir('..')
papers = pd.read_csv('/content/db_supplychain_merged-title-changed-dropped-replaced-phrases.csv')
print(papers.head(5))

                                     Author Keywords  \
0  ADDITIVE MANUFACTURING;  AUTOMATION;  ELOGISTI...   
1  DIGITALTWIN;  MILITARY SUPPLYCHAIN;  MONITORIN...   
2  BLOCKCHAIN;  METAVERSE;  RESILIENCE;  SUPPLYCHAIN   
3  BUSINESS PROCESS OPTIMIZATION;  DIGITALSUPPLYC...   
4  DIGITALSUPPLYCHAINTWIN;  DIGITALTWINS;  DISCRE...   

                                            Abstract  \
0  THE PRIMARY OBJECTIVE OF THIS PAPER IS TO IDEN...   
1  THE DYNAMIC AND COMPLEX NATURE OF THE MILITARY...   
2  METAVERSES ARE GETTING MUCH MORE ATTENTION IN ...   
3  WE PRESENT AN INTEGRATED DIGITALTWIN FRAMEWORK...   
4  THE DEVELOPMENT OF NEW INFORMATION TECHNOLOGIE...   

                                     Source Title  \
0         LECTURE NOTES IN MECHANICAL ENGINEERING   
1       ADVANCES IN TRANSDISCIPLINARY ENGINEERING   
2  ACM INTERNATIONAL CONFERENCE PROCEEDING SERIES   
3              COMPUTERS AND CHEMICAL ENGINEERING   
4                MOBILE NETWORKS AND APPLICATIONS   

       

In [ ]:
# Load the regular expression library
import re

**Remove puctuations and convert data to lowercase**

In [ ]:
# Remove punctuation
papers['paper_text_processed'] = papers['Abstract'].map(lambda x: re.sub('[,\.!?]', '', str(x)))
papers['paper_text_processed'] = papers['paper_text_processed'].map(lambda x: re.sub('[,\.!?]', '', x))

# Convert the abstracts to lowercase
papers['paper_text_processed'] = papers['paper_text_processed'].map(lambda x: x.lower())

In [ ]:
import gensim
from gensim.utils import simple_preprocess

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
       yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc = de-accents

data = papers.paper_text_processed.values.tolist()
data_words = list(sent_to_words(data))

print(data_words)

[['the', 'primary', 'objective', 'of', 'this', 'paper', 'is', 'to', 'identify', 'how', 'industrial', 'automation', 'of', 'additive', 'manufacturing', 'technology', 'using', 'internet', 'of', 'things', 'iot', 'will', 'leverage', 'the', 'quality', 'of', 'existing', 'supplychain', 'management', 'processes', 'the', 'importance', 'of', 'the', 'implementation', 'of', 'elogistics', 'in', 'is', 'also', 'delineated', 'an', 'intensive', 'desk', 'based', 'literature', 'survey', 'study', 'identifies', 'potential', 'advantages', 'of', 'incorporating', 'iot', 'automated', 'additive', 'manufacturing', 'to', 'improve', 'supplychain', 'processes', 'we', 'also', 'discuss', 'how', 'the', 'automation', 'of', 'logistics', 'operations', 'also', 'known', 'as', 'elogistics', 'will', 'help', 'modernize', 'supplychain', 'management', 'the', 'research', 'paper', 'comprises', 'theoretical', 'work', 'about', 'the', 'application', 'of', 'automated', 'additive', 'manufacturing', 'and', 'elogistics', 'in', 'modern', 

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [ ]:
# NLTK Stop words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
!python -m spacy download en_core_web_sm
import spacy

2023-08-11 10:41:02.266267: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-11 10:41:03.404185: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 76.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized)

[['primary', 'objective', 'paper', 'identify', 'industrial', 'automation', 'additive', 'manufacture', 'technology', 'use', 'internet', 'thing', 'leverage', 'quality', 'exist', 'supplychain', 'management', 'process', 'importance', 'implementation', 'elogistic', 'also', 'delineate', 'intensive', 'desk', 'base', 'survey', 'study', 'identify', 'potential', 'advantage', 'incorporate', 'iot', 'automate', 'additive', 'manufacturing', 'improve', 'supplychain', 'process', 'also', 'discuss', 'automation', 'logistic', 'operation', 'also', 'know', 'elogistic', 'help', 'modernize', 'supplychain', 'management', 'research', 'paper', 'comprise', 'theoretical', 'work', 'application', 'automate', 'additive', 'manufacture', 'elogistic', 'modern', 'detailed', 'analysis', 'advantage', 'implement', 'automate', 'additive', 'manufacturing', 'technology', 'elogistic', 'supplychain', 'perform', 'due', 'novelty', 'area', 'conduct', 'study', 'base', 'available', 'research', 'material', 'pretty', 'limited', 'vario

In [ ]:
import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus)
print(len(corpus))

[[(0, 4), (1, 2), (2, 3), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 3), (10, 4), (11, 1), (12, 2), (13, 1), (14, 1), (15, 2), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 4), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 2), (32, 1), (33, 1), (34, 1), (35, 1), (36, 2), (37, 2), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 2), (50, 2), (51, 2), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 2), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 3), (69, 1), (70, 1), (71, 1), (72, 1), (73, 2), (74, 1), (75, 1), (76, 1), (77, 1), (78, 2), (79, 6), (80, 1), (81, 3), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1)], [(4, 1), (7, 1), (8, 2), (20, 2), (21, 2), (28, 1), (31, 1), (33, 1), (35, 2), (40, 1), (48, 1), (49, 2), (58, 1), (62, 2), (71, 1), (79, 3), (81, 4), (83, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (

In [ ]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10,
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

In [ ]:
from pprint import pprint

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.026*"industryfourth" + 0.023*"technology" + 0.022*"scre" + 0.019*"effect" '
  '+ 0.019*"ripple" + 0.015*"disruption" + 0.015*"kpi" + 0.011*"resilience" + '
  '0.011*"improve" + 0.011*"supplychain"'),
 (1,
  '0.001*"technology" + 0.001*"industryfourth" + 0.001*"method" + '
  '0.001*"supplychain" + 0.001*"digitalization" + 0.001*"control" + '
  '0.001*"digitaltwin" + 0.001*"model" + 0.001*"smart" + 0.001*"new"'),
 (2,
  '0.028*"end" + 0.023*"ensure" + 0.019*"technology" + 0.019*"sc" + '
  '0.019*"visibility" + 0.014*"supplychain" + 0.014*"enable" + '
  '0.014*"effective" + 0.014*"monitoring" + 0.010*"digitaltwin"'),
 (3,
  '0.017*"supplychain" + 0.015*"technology" + 0.013*"sc" + 0.013*"application" '
  '+ 0.013*"study" + 0.012*"manufacturing" + 0.009*"risk" + 0.009*"dsct" + '
  '0.008*"development" + 0.008*"information"'),
 (4,
  '0.020*"model" + 0.020*"production" + 0.018*"disruption" + '
  '0.015*"supplychain" + 0.013*"digitaltwin" + 0.013*"datum" + 0.013*"risk" + '
  '0.011*

In [ ]:
from gensim.models import CoherenceModel

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.40666394020915275


In [ ]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):

    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k,
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)

    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')

    return coherence_model_lda.get_coherence()

In [ ]:
import numpy as np
import tqdm

grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.5))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.5))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)),
               corpus]

corpus_title = ['75% Corpus', '100% Corpus']

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=(len(beta)*len(alpha)*len(topics_range)*len(corpus_title)))

    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word,
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)

                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('/content/lda_tuning_results_supply_chain.csv', index=False)
    pbar.close()

100%|██████████| 216/216 [03:29<00:00,  1.03it/s]


In [ ]:
num_topics = 6
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics,
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha='asymmetric',
                                           eta='symmetric')

In [ ]:
!pip install --upgrade gensim
!pip install pyldavis==3.4.1
import pyLDAvis.gensim
import pickle
import pyLDAvis

In [ ]:
pyLDAvis.enable_notebook()

LDAvis_data_filepath = os.path.join('/content/ldavis_prepared_' + str(num_topics))

if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

pyLDAvis.save_html(LDAvis_prepared, '/content/ldavis_prepared_' + str(num_topics) + '.html')

LDAvis_prepared

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.009735  0.153467       1        1  30.891899
0     -0.022216 -0.025840       2        1  28.882152
5     -0.143864 -0.043056       3        1  23.135103
2      0.099092 -0.039416       4        1   8.636604
3      0.062751 -0.015118       5        1   5.678365
1      0.013973 -0.030038       6        1   2.775878, topic_info=           Term       Freq      Total Category  logprob  loglift
233  simulation  12.000000  12.000000  Default  30.0000  30.0000
81   technology  25.000000  25.000000  Default  29.0000  29.0000
148          sc  12.000000  12.000000  Default  28.0000  28.0000
69   production  13.000000  13.000000  Default  27.0000  27.0000
530     benefit   6.000000   6.000000  Default  26.0000  26.0000
..          ...        ...        ...      ...      ...      ...
303      method   0.592263  11.947686   Topic6  -4.8059   0.5799
85          use   0.607602  15.194629   Topic6  -4.7804   0.3650
78        study   0.500108  15.692986   Topic6  -4.9751   0.1381
617    simulate   0.394240   3.664664   Topic6  -5.2129   1.3547
540        gain   0.375680   2.690566   Topic6  -5.2612   1.6154

[360 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
527       2  0.539502    academia
359       1  0.749563    adaptive
0         1  0.620537    additive
0         2  0.206846    additive
240       2  0.306226     address
...     ...       ...         ...
157       2  0.178744  visibility
157       4  0.536233  visibility
158       4  0.794231     weather
159       4  0.794236       whole
160       4  0.794255    wireless

[458 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 1, 6, 3, 4, 2])

In [ ]:
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[1] if ldamodel.per_word_topics else row_list
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data_lemmatized)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

#Export file
df_export_1 = pd.DataFrame(df_dominant_topic) # Dominant topic and its percentage contribution in each document
df_export_1.to_csv('/content/df_ssuplychain_export_dominant_topic_perc_contrib.csv', index=False, header=True)

#Show
df_dominant_topic

<ipython-input-16-e75a8fdabc70>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
<ipython-input-16-e75a8fdabc70>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
<ipython-input-16-e75a8fdabc70>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
<ipython-input-16-e75a8fdabc70>:15: FutureWarning: The frame.append method is deprecated and will be removed from p

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,4,0.8480,"process, model, supplychain, digitaltwin, prod...","[primary, objective, paper, identify, industri..."
1,1,2,0.9934,"technology, enable, end, supplychain, sc, ensu...","[dynamic, complex, nature, military, supplycha..."
2,2,5,0.7284,"supplychain, digitaltwin, use, management, met...","[metaverse, get, much, attention, industry, fu..."
3,3,4,0.9892,"process, model, supplychain, digitaltwin, prod...","[present, integrate, digitaltwin, framework, s..."
4,4,0,0.9919,"technology, application, digitaltwin, supplych...","[development, new, information, technology, be..."
5,5,0,0.8910,"technology, application, digitaltwin, supplych...","[digitaltwin, integration, virtual, physical, ..."
6,6,5,0.9899,"supplychain, digitaltwin, use, management, met...","[management, perishable, food, inventory, dema..."
7,7,4,0.9897,"process, model, supplychain, digitaltwin, prod...","[paper, explore, current, role, future, potent..."
8,8,5,0.9846,"supplychain, digitaltwin, use, management, met...","[object, intensive, interest, research, practi..."
9,9,0,0.8024,"technology, application, digitaltwin, supplych...","[paper, examine, implementation, digitaltwin, ..."


In [ ]:
# Display setting to show more characters in column
pd.options.display.max_colwidth = 100

sent_topics_sorteddf_mallet = pd.DataFrame()
sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet,
                                             grp.sort_values(['Perc_Contribution'],
                                             ascending=False).head(1)],
                                             axis=0)

# Reset Index
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Representative Text"]

# Export
df_export_2 = pd.DataFrame(sent_topics_sorteddf_mallet) # The most representative sentence for each topic
df_export_2.to_csv('/content/df_supplychain_export_most_repre_sentence.csv', index=False, header=True)

#Show
sent_topics_sorteddf_mallet

,Topic_Num,Topic_Perc_Contrib,Keywords,Representative Text
0,0,0.9941,"technology, application, digitaltwin, supplychain, system, study, paper, platform, product, review","[continue, advancement, compute, digital, technology, transform, market, economic, business, dis..."
1,1,0.6207,"quantify, benefit, simulation, performance, experiment, organic, lscm, use, method, practice","[background, gain, increase, attention, academia, practice, positive, impact, logistic, supplych..."
2,2,0.9934,"technology, enable, end, supplychain, sc, ensure, visibility, effective, monitoring, digital","[dynamic, complex, nature, military, supplychain, msc, require, effective, monitoring, supply, t..."
3,3,0.9888,"sc, mp, new, technology, production, conceptual, algorithm, propose, environment, model","[recently, many, novel, paradigms, concept, technology, lay, foundation, new, revolution, manufa..."
4,4,0.9925,"process, model, supplychain, digitaltwin, production, framework, datum, disruption, optimization...","[background, internet, thing, become, prevalent, recent, year, digitaltwin, attract, lot, attent..."
5,5,0.9928,"supplychain, digitaltwin, use, management, method, research, digitalization, provide, quality, m...","[purpose, research, consist, improve, theoretical, basis, non, destructive, quality, control, me..."


In [ ]:
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()
topic_contribution = round(topic_counts/topic_counts.sum(), 4)
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

vol_dis_of_topic = pd.concat(
   [topic_num_keywords, topic_counts, topic_contribution], axis=1
)

vol_dis_of_topic.columns = [
   'Dominant-Topic', 'Topic-Keywords', 'Num_Documents', 'Perc_Documents'
]

# Export
df_export_2 = pd.DataFrame(vol_dis_of_topic) # Volume & distribution of topics
df_export_2.to_csv('/content/df_supplychain_export_vol_distribution.csv', index=False, header=True)

# Show
vol_dis_of_topic

,Dominant-Topic,Topic-Keywords,Num_Documents,Perc_Documents
0,4,"process, model, supplychain, digitaltwin, production, framework, datum, disruption, optimization...",8.0,0.2963
1,2,"technology, enable, end, supplychain, sc, ensure, visibility, effective, monitoring, digital",1.0,0.0370
2,5,"supplychain, digitaltwin, use, management, method, research, digitalization, provide, quality, m...",2.0,0.0741
3,4,"process, model, supplychain, digitaltwin, production, framework, datum, disruption, optimization...",2.0,0.0741
4,0,"technology, application, digitaltwin, supplychain, system, study, paper, platform, product, review",8.0,0.2963
5,0,"technology, application, digitaltwin, supplychain, system, study, paper, platform, product, review",6.0,0.2222
6,5,"supplychain, digitaltwin, use, management, method, research, digitalization, provide, quality, m...",NaN,NaN
7,4,"process, model, supplychain, digitaltwin, production, framework, datum, disruption, optimization...",NaN,NaN
8,5,"supplychain, digitaltwin, use, management, method, research, digitalization, provide, quality, m...",NaN,NaN
9,0,"technology, application, digitaltwin, supplychain, system, study, paper, platform, product, review",NaN,NaN


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
doc_lens = [len(d) for d in df_dominant_topic.Text]

# Plot
plt.figure(figsize=(16,7), dpi=160)
plt.hist(doc_lens, bins = 1000, color='navy')
plt.text(750, 100, "Mean   : " + str(round(np.mean(doc_lens))))
plt.text(750,  90, "Median : " + str(round(np.median(doc_lens))))
plt.text(750,  80, "Stdev   : " + str(round(np.std(doc_lens))))
plt.text(750,  70, "1%ile    : " + str(round(np.quantile(doc_lens, q=0.01))))
plt.text(750,  60, "99%ile  : " + str(round(np.quantile(doc_lens, q=0.99))))

plt.gca().set(xlim=(0, 1000), ylabel='Number of Documents', xlabel='Document Word Count')
plt.tick_params(size=16)
plt.xticks(np.linspace(0,1000,9))
plt.title('Distribution of Document Word Counts', fontdict=dict(size=22))
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.colors as mcolors

In [ ]:
cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]  # more colors: 'mcolors.XKCD_COLORS'

fig, axes = plt.subplots(3,3,figsize=(16,14), dpi=160, sharex=True, sharey=True)

for i, ax in enumerate(axes.flatten()):
    df_dominant_topic_sub = df_dominant_topic.loc[df_dominant_topic.Dominant_Topic == i, :]
    doc_lens = [len(d) for d in df_dominant_topic_sub.Text]
    ax.hist(doc_lens, bins = 1000, color=cols[i])
    ax.tick_params(axis='y', labelcolor=cols[i], color=cols[i])
    sns.kdeplot(doc_lens, color="black", shade=False, ax=ax.twinx())
    ax.set(xlim=(0, 1000), xlabel='Document Word Count')
    ax.set_ylabel('Number of Documents', color=cols[i])
    ax.set_title('Topic: '+str(i), fontdict=dict(size=16, color=cols[i]))

fig.tight_layout()
fig.subplots_adjust(top=0.90)
plt.xticks(np.linspace(0,1000,9))
fig.suptitle('Distribution of Document Word Counts by Dominant Topic', fontsize=22)
plt.show()

NameError: ignored